# Accessing CLMS API
https://eea.github.io/clms-api-docs/download.html

## vSensor

In [51]:
import requests
import json
import xarray as xr
import rasterio as rio
import osgeo
import time
import jwt

API="https://land.copernicus.eu/api/"
VERSION="v1"
PRODUCT="CORINE-Land-Cover"

# Load saved key from filesystem
service_key = json.load(open('../references/corine/clc.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)


emCCV3IdSvt3yaAs-r7Jkh5D73CibTKMUOJdjGQUhBCf6Jsa3JQjz3bZUJbJYFgeWsFhkf0eSxxrQRLrtwn5Rw==


In [22]:
req1= requests.post(f'{API}@@oauth2-token', headers={'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'}, data='grant_type=urn%3Aietf%3Aparams%3Aoauth%3Agrant-type%3Ajwt-bearer&assertion=REDACTED')
req1.json()
req2= requests.get(f'{API}@search?portal_type=DataSet', headers={'Accept': 'application/json', 'Authorization': 'Bearer <REDACTED>'})
req2.json()

TypeError: Session.request() got an unexpected keyword argument 'b_size'

In [23]:
req3=requests.get(f'{API}@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=dataset_full_format&&metadata_fields=dataset_download_information&&b_size=100000', headers={'Accept': 'application/json'})
req3.json()

{'@id': 'https://land.copernicus.eu/api/@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=dataset_full_format&metadata_fields=dataset_download_information',
 'items': [{'@id': 'https://land.copernicus.eu/api/en/products/vegetation/gross-primary-production-v1-0-300m',
   '@type': 'DataSet',
   'UID': 'bdaa99b9878c4eb4a8cf481f4a414ba0',
   'dataset_download_information': {'items': []},
   'dataset_full_format': 'Netcdf',
   'description': 'The total amount of carbon compounds produced by photosynthesis of plants in an ecosystem in a given period of time.',
   'getRemoteUrl': None,
   'image_field': '',
   'image_scales': None,
   'review_state': 'published',
   'title': 'Gross Primary Production 2023-present (raster 300 m), global, 10-daily – version 1',
   'type_title': 'DataSet'},
  {'@id': 'https://land.copernicus.eu/api/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2019-globe',
   '@type': 'DataSet',
   'UID': 'a

In [24]:
with open('../references/corine/clms-docs.json', 'w') as f:
    json.dump(req3.json(), f)

In [32]:
clc = req3.json()['items']
substrings = ['CORINE Land Cover', 'Europe', '100 m', '6-yearly']
for data in clc:
    if (substrings[0] in data['title']) & (substrings[2] in data['title']) & (substrings[3] in data['title']) & ('Change' not in data['title']):
        print(data['title'])
        print(data['dataset_full_format'])
        print(data['dataset_download_information'])
        print(data['UID'])
        print(data['@id'])

CORINE Land Cover 2018 (vector/raster 100 m), Europe, 6-yearly
None
{'items': [{'@id': '1bda2fbd-3230-42ba-98cf-69c96ac063bc', 'full_format': 'GDB', 'full_path': 'H:\\Corine_Land_Cover\\Corine_Land_Cover\\Corine_Land_Cover(CLC)_2018\\CLC2018_Version_20\\Data\\data-master\\U2018_CLC2018_V2020_20u1_vector\\U2018_CLC2018_V2020_20u1.gdb', 'full_source': 'EEA', 'name': 'VECTOR'}, {'@id': '7bcdf9d1-6ba0-4d4e-afa8-01451c7316cb', 'collection': '100 m', 'full_format': 'Geotiff', 'full_path': 'H:\\Corine_Land_Cover\\Corine_Land_Cover\\Corine_Land_Cover(CLC)_2018\\CLC2018_Version_20\\Data\\data-master\\U2018_CLC2018_V2020_20u1_raster100m_tiled', 'full_source': 'EEA', 'name': 'RASTER'}]}
0407d497d3c44bcd93ce8fd5bf78596a
https://land.copernicus.eu/api/en/products/corine-land-cover/clc2018
CORINE Land Cover 2012 (vector/raster 100 m), Europe, 6-yearly
GDB
{'items': [{'@id': 'cff14ee5-bafb-46f4-a1b2-2cd6f4049514', 'full_format': {'title': 'GDB', 'token': 'GDB'}, 'full_path': 'H:\\Corine_Land_Cover\\C

In [42]:
import glob
import os

current =[]
new = []
obj = {}
for data in clc:
        if (substrings[0] in data['title']) & (substrings[2] in data['title']) & (substrings[3] in data['title']) & ('Change' not in data['title']):
            obj['title']=data['title']
            obj['dataset_full_format']=data['dataset_full_format']
            obj['dataset_download_information']=data['dataset_download_information']
            obj['UID']=data['UID']
            obj['@id']=data['@id']
            current.append(obj)
            obj = {}

list_of_files = glob.glob('../logs/feedback/corine/*.json') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
    
if latest_file:
        with open(latest_file, 'r') as f:
            old = json.load(f)
            for i in new:
                if i not in old:
                    new.append(i) 
                    print('new CORINE data available...')
                else:
                    print('no new data')
            with open(f'../logs/feedback/corine/clms-docs-{round(time.time())}.json', 'w') as f:
                json.dump(new, f)
    


## Intaker

In [52]:
for i in current:
    DatasetID = i['UID']
    DatasetDownloadInformationID = ''

    for x in i['dataset_download_information']['items']:
        if x['full_format'] == 'Geotiff':
            DatasetDownloadInformationID = x['@id']
    print(DatasetID)
    print(DatasetDownloadInformationID)           
    task=requests.post(f'{API}@datarequest_post', headers={'Accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': f'Bearer {access_token}'}, json={'Datasets': [{'DatasetID': DatasetID, 'DatasetDownloadInformationID': DatasetDownloadInformationID, 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326'}]})
    print(task.json())

0407d497d3c44bcd93ce8fd5bf78596a
7bcdf9d1-6ba0-4d4e-afa8-01451c7316cb
{'ErrorTaskIds': [], 'TaskIds': [{'TaskID': '40569095338'}]}
a5ee71470be04d66bcff498f94ceb5dc
b1027c88-6256-435b-98a3-62f019cbb381
{'ErrorTaskIds': [], 'TaskIds': [{'TaskID': '77840732945'}]}
d443c86fec2f49e08ff12c7decdbf2af
4841f8cb-6e7c-4227-bb35-cd787609c546
{'ErrorTaskIds': [], 'TaskIds': [{'TaskID': '90187946113'}]}
6704f90ca82e4f228a46111519f8978e
19d7fbd8-53ee-4d09-89b6-fe5231247763
{'ErrorTaskIds': [], 'TaskIds': [{'TaskID': '6040838668'}]}
8b393c8e474642d5acb4ed70fc2dc024
fecff4c7-a5e5-428d-817d-a8ee5f23d99d
{'ErrorTaskIds': [], 'TaskIds': [{'TaskID': '87825280935'}]}


In [58]:
status=requests.get(f'{API}@datarequest_search', headers={'Accept': 'application/json', 'Content-Type': 'application/json', 'Authorization': f'Bearer {access_token}'})
print(status.json())
with open('../references/corine/status-sample.json', 'w') as f:
    json.dump(status.json(), f)

{'40569095338': {'Datasets': [{'DatasetFormat': 'Geotiff', 'DatasetID': '0407d497d3c44bcd93ce8fd5bf78596a', 'DatasetPath': 'SDpcQ29yaW5lX0xhbmRfQ292ZXJcQ29yaW5lX0xhbmRfQ292ZXJcQ29yaW5lX0xhbmRfQ292ZXIoQ0xDKV8yMDE4XENMQzIwMThfVmVyc2lvbl8yMFxEYXRhXGRhdGEtbWFzdGVyXFUyMDE4X0NMQzIwMThfVjIwMjBfMjB1MV9yYXN0ZXIxMDBtX3RpbGVk', 'DatasetSource': 'EEA', 'DatasetTitle': 'CORINE Land Cover 2018 (vector/raster 100 m), Europe, 6-yearly', 'Metadata': ['https://sdi.eea.europa.eu/catalogue/copernicus/api/records/960998c1-1870-4e82-8051-6485205ebbac/formatters/xml?approved=true', 'https://sdi.eea.europa.eu/catalogue/copernicus/api/records/71c95a07-e296-44fc-b22b-415f42acfdf0/formatters/xml?approved=true'], 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'WekeoChoices': ''}], 'FMETaskId': 25480, 'RegistrationDateTime': '2023-11-13T14:14:01.323884', 'Status': 'Queued', 'UserID': 'n00eckgi'}, '6040838668': {'Datasets': [{'DatasetFormat': 'Geotiff', 'DatasetID': '6704f90ca82e4f228a46111519f8978e', 'Datase